In [8]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import time
import re
import random

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit, resp.request.url)
            time.sleep(random.random()*5) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

### 네이버뉴스 수집 - 200701부터 6개분야에서 top30

In [9]:
import pandas as pd
pd.to_datetime('20200801', format='%Y%m%d')

Timestamp('2020-08-01 00:00:00')

In [10]:
from datetime import datetime, timedelta 

start_date = datetime.strptime('20180731', '%Y%m%d') 
end_date = datetime.today()
str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

In [11]:
str_date_list[:3], str_date_list[-3:] #날짜확인

(['20180731', '20180801', '20180802'], ['20200805', '20200806', '20200807'])

### 링크수집

In [26]:
urlst = []
for i in range(6):
    for j in str_date_list[:2]: #just for test
        url = 'https://news.naver.com/main/ranking/popularDay.nhn'

        params = {
            'rankingType': 'popular_day',
        }

        params['sectionId'] = '10'+str(i)
        params['date']=j

        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }

        resp = download(url, params=params, headers=headers)
        dom = BeautifulSoup(resp.content, 'lxml')


        #뉴스링크 수집
        for _ in dom.find_all('a',{'class':'nclicks(rnk.eco)'}):
            if _['href'] not in urlst:
                urlst.append(_['href'])


In [27]:
urlst[:3] #url확인

['/main/ranking/read.nhn?rankingType=popular_day&oid=001&aid=0010244310&date=20180731&type=1&rankingSectionId=101&rankingSeq=1',
 '/main/ranking/read.nhn?rankingType=popular_day&oid=028&aid=0002419466&date=20180731&type=1&rankingSectionId=101&rankingSeq=2',
 '/main/ranking/read.nhn?rankingType=popular_day&oid=015&aid=0003988450&date=20180731&type=1&rankingSectionId=101&rankingSeq=4']

In [31]:
#get html from article address
for _ in urlst:
    newslink = urljoin(url, _)
    resp = download(newslink, headers=headers)
    if resp.status_code!=200:
        continue
    dom = BeautifulSoup(resp.content, 'html.parser')
    break

### 제목스크래핑

In [32]:
dom.find('h3', {'id':'articleTitle'}).text.strip()

'리콜 대상 BMW 420d 주행 중 또 화재…이번엔 경인고속도로'

### 기사작성일

In [33]:
dom.find_all('span', {'class':'t11'})[0].text.strip()

'2018.07.31. 오후 5:57'

### 본문스크래핑

In [34]:
dom.find_all('div',{'id':'articleBodyContents'})[0].text.strip()

'// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n경인고속도로 달리던 리콜대상 BMW 차량서 불(인천=연합뉴스) 31일 오후 4시 26분께 인천시 서구 경인고속도로 서울 방향 가좌IC 인근에서 주행 중인 BMW 420d 차량에 불이 붙어 소방당국이 진화했다. 불이 난 차량은 최근 BMW코리아가 조치한 리콜대상에 포함된 차종이다.2018.7.31   [인천 서부소방서 제공]BMW 10만6,000여대 리콜…27일부터 안전진단(CG)[연합뉴스TV 제공](인천=연합뉴스) 최은지 기자 = 최근 잇따른 주행 중 화재 사고로 리콜(시정명령) 결정이 내려진 BMW 차량에서 운행 중 또 불이 났다.    31일 인천 서부소방서에 따르면 이날 오후 4시 26분께 인천시 서구 경인고속도로 서울 방향 가좌IC 인근에서 주행 중인 420d 차량에 불이 붙었다.    이 차량 운전자는 소방당국에 "엔진룸에서 불이 났다"고 진술했다.    화재 당시 운전자 등 2명이 차량에 타고 있었으나 모두 신속히 대피해 인명피해는 없었다.    불이 난 차량은 2014년식 BMW 420d로, 최근 BMW코리아가 조치한 리콜대상에 포함된 차종이다.    소방당국은 신고를 받고 출동해 9분 만인 이날 오후 4시 35분께 불을 모두 껐다.    BMW코리아는 이달 26일 BMW 520d 등 총 42개 차종 10만6천317대를 대상으로 자발적 리콜조치를 한다고 밝혔다.    BMW는 27일부터 해당 차량 전체에 대해 긴급 안전진단을 벌이고 있으며 8월 중순부터 엔진에 장착된 배기가스 재순환장치(EGR) 모듈 개선품 교체를 본격 진행할 예정이다.    BMW 측의 자발적인 리콜 조치가 내려진 상황에서도 차량 화재가 잇따르자 BMW 운전자들은 불안을 호소했다.    전날 낮 12시께 인천시 서구 수도권 제2외곽순환고속도로 인천∼김포 구간(인천김포고속도로) 내 북항터널에서도 달리던 BMW GT 차량에 불이 붙었다. 당일 불이 난 2013년식

In [37]:
txt = dom.find_all('div',{'id':'articleBodyContents'})[0].text.replace(
            '\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n','').replace(
            '<br/>','').strip()

txt = txt[:re.search("▶", txt).start()]
txt

'경인고속도로 달리던 리콜대상 BMW 차량서 불(인천=연합뉴스) 31일 오후 4시 26분께 인천시 서구 경인고속도로 서울 방향 가좌IC 인근에서 주행 중인 BMW 420d 차량에 불이 붙어 소방당국이 진화했다. 불이 난 차량은 최근 BMW코리아가 조치한 리콜대상에 포함된 차종이다.2018.7.31   [인천 서부소방서 제공]BMW 10만6,000여대 리콜…27일부터 안전진단(CG)[연합뉴스TV 제공](인천=연합뉴스) 최은지 기자 = 최근 잇따른 주행 중 화재 사고로 리콜(시정명령) 결정이 내려진 BMW 차량에서 운행 중 또 불이 났다.    31일 인천 서부소방서에 따르면 이날 오후 4시 26분께 인천시 서구 경인고속도로 서울 방향 가좌IC 인근에서 주행 중인 420d 차량에 불이 붙었다.    이 차량 운전자는 소방당국에 "엔진룸에서 불이 났다"고 진술했다.    화재 당시 운전자 등 2명이 차량에 타고 있었으나 모두 신속히 대피해 인명피해는 없었다.    불이 난 차량은 2014년식 BMW 420d로, 최근 BMW코리아가 조치한 리콜대상에 포함된 차종이다.    소방당국은 신고를 받고 출동해 9분 만인 이날 오후 4시 35분께 불을 모두 껐다.    BMW코리아는 이달 26일 BMW 520d 등 총 42개 차종 10만6천317대를 대상으로 자발적 리콜조치를 한다고 밝혔다.    BMW는 27일부터 해당 차량 전체에 대해 긴급 안전진단을 벌이고 있으며 8월 중순부터 엔진에 장착된 배기가스 재순환장치(EGR) 모듈 개선품 교체를 본격 진행할 예정이다.    BMW 측의 자발적인 리콜 조치가 내려진 상황에서도 차량 화재가 잇따르자 BMW 운전자들은 불안을 호소했다.    전날 낮 12시께 인천시 서구 수도권 제2외곽순환고속도로 인천∼김포 구간(인천김포고속도로) 내 북항터널에서도 달리던 BMW GT 차량에 불이 붙었다. 당일 불이 난 2013년식 BMW GT도 리콜대상에 포함된 차종이었다.    chamse@yna.co.kr '

### 요약본 스크래핑

In [38]:
newslink =  urljoin(url, urlst[0]) #for test
parsed = (urlparse(newslink)).query.split('&')
oid = parsed[1]
aid = parsed[2]
print(oid, aid)

oidno = re.search(re.compile(r'[^oid=]{3}'), oid).group(0)
print(oidno)

aidno = re.search(re.compile(r'[^aid=]{10}'), aid).group(0)
print(aidno)

oid=001 aid=0010244310
001
0010244310


In [40]:
sumurl = 'https://tts.news.naver.com/article/'+oidno+'/'+aidno+'/summary'
sumresp = download(sumurl)
json = sumresp.json()
json['summary']

'최근 잇따른 주행 중 화재 사고로 리콜 결정이 내려진 BMW 차량에서 운행 중 또 불이 났다.<br/><br/>전날 낮 12시께 인천시 서구 수도권 제2외곽순환고속도로 인천∼김포 구간 내 북항터널에서도 달리던 BMW GT 차량에 불이 붙었다.<br/><br/>당일 불이 난 2013년식 BMW GT도 리콜대상에 포함된 차종이었다.'

In [41]:
summary = json['summary'].replace('<br/>','')
summary

'최근 잇따른 주행 중 화재 사고로 리콜 결정이 내려진 BMW 차량에서 운행 중 또 불이 났다.전날 낮 12시께 인천시 서구 수도권 제2외곽순환고속도로 인천∼김포 구간 내 북항터널에서도 달리던 BMW GT 차량에 불이 붙었다.당일 불이 난 2013년식 BMW GT도 리콜대상에 포함된 차종이었다.'

### combined

In [43]:
for _ in urlst:
    newslink = urljoin(url, _)
    resp = download(newslink, headers=headers)
    if resp.status_code!=200:
        continue
    dom = BeautifulSoup(resp.content, 'html.parser')
    if dom.find_all('a',{'class':'media_end_head_autosummary_button _toggle_btn nclicks(sum_summary)'}) != []:
        #제목
        title = dom.find('h3', {'id':'articleTitle'}).text.strip()
        #기사작성일
        date = dom.find_all('span', {'class':'t11'})[0].text.strip()
        #본문
        body = dom.find_all('div',{'id':'articleBodyContents'})[0].text.replace(
            '\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n','').replace(
            '<br/>','').strip()
        
        if re.search("▶", body):
            body = body[:re.search("▶", body).start()]

        parsed = (urlparse(newslink)).query.split('&')
        oid = parsed[1]
        aid = parsed[2]

        oidno = re.search(re.compile(r'[^oid=]{3}'), oid).group(0)
        aidno = re.search(re.compile(r'[^aid=]{10}'), aid).group(0)

        sumurl = 'https://tts.news.naver.com/article/'+oidno+'/'+aidno+'/summary'
        sumresp = download(sumurl)
        if sumresp.status_code!=200:
            continue
        json = sumresp.json()
        #요약
        summary = json['summary'].replace('<br/>','')

        print(title)
        print(date)
        print(body)
        print(summary)
        break

리콜 대상 BMW 420d 주행 중 또 화재…이번엔 경인고속도로
2018.07.31. 오후 5:57
경인고속도로 달리던 리콜대상 BMW 차량서 불(인천=연합뉴스) 31일 오후 4시 26분께 인천시 서구 경인고속도로 서울 방향 가좌IC 인근에서 주행 중인 BMW 420d 차량에 불이 붙어 소방당국이 진화했다. 불이 난 차량은 최근 BMW코리아가 조치한 리콜대상에 포함된 차종이다.2018.7.31   [인천 서부소방서 제공]BMW 10만6,000여대 리콜…27일부터 안전진단(CG)[연합뉴스TV 제공](인천=연합뉴스) 최은지 기자 = 최근 잇따른 주행 중 화재 사고로 리콜(시정명령) 결정이 내려진 BMW 차량에서 운행 중 또 불이 났다.    31일 인천 서부소방서에 따르면 이날 오후 4시 26분께 인천시 서구 경인고속도로 서울 방향 가좌IC 인근에서 주행 중인 420d 차량에 불이 붙었다.    이 차량 운전자는 소방당국에 "엔진룸에서 불이 났다"고 진술했다.    화재 당시 운전자 등 2명이 차량에 타고 있었으나 모두 신속히 대피해 인명피해는 없었다.    불이 난 차량은 2014년식 BMW 420d로, 최근 BMW코리아가 조치한 리콜대상에 포함된 차종이다.    소방당국은 신고를 받고 출동해 9분 만인 이날 오후 4시 35분께 불을 모두 껐다.    BMW코리아는 이달 26일 BMW 520d 등 총 42개 차종 10만6천317대를 대상으로 자발적 리콜조치를 한다고 밝혔다.    BMW는 27일부터 해당 차량 전체에 대해 긴급 안전진단을 벌이고 있으며 8월 중순부터 엔진에 장착된 배기가스 재순환장치(EGR) 모듈 개선품 교체를 본격 진행할 예정이다.    BMW 측의 자발적인 리콜 조치가 내려진 상황에서도 차량 화재가 잇따르자 BMW 운전자들은 불안을 호소했다.    전날 낮 12시께 인천시 서구 수도권 제2외곽순환고속도로 인천∼김포 구간(인천김포고속도로) 내 북항터널에서도 달리던 BMW GT 차량에 불이 붙었다. 당일 불이 난 2013년식 BMW GT도 리콜대

### DB에 넣기

In [29]:
import sqlite3

conn = sqlite3.connect('news.db')
cur = conn.cursor()

In [30]:
cur.executescript('''
    DROP TABLE IF EXISTS navernews;
    CREATE TABLE navernews(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        date TEXT,
        body TEXT,
        summary TEXT,
        link TEXT NOT NULL
    );
''')

In [31]:
for j in str_date_list:
    for i in range(6):
        print(j,i)
        url = 'https://news.naver.com/main/ranking/popularDay.nhn'

        params = {
            'rankingType': 'popular_day',
        }

        params['sectionId'] = '10'+str(i)
        params['date']=j

        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }

        resp = download(url, params=params, headers=headers)
        if resp.status_code!=200:
            continue
        dom = BeautifulSoup(resp.content, 'lxml')
        
        
        #뉴스링크 수집
        urlst = []
        for _ in dom.find_all('div',{'class':'ranking_headline'}):
            if _.a['href'] not in urlst:
                urlst.append(_.a['href'])
        
        for _ in urlst:
            newslink = urljoin(url, _)
            resp = download(newslink, headers=headers)
            if resp.status_code!=200:
                continue
            dom = BeautifulSoup(resp.content, 'html.parser')
            if dom.find_all('a',{'class':'media_end_head_autosummary_button _toggle_btn nclicks(sum_summary)'}) != []:
                #제목
                title = dom.find('h3', {'id':'articleTitle'}).text.strip()
                #기사작성일
                date = dom.find_all('span', {'class':'t11'})[0].text.strip()
                #본문
                body = dom.find_all('div',{'id':'articleBodyContents'})[0].text.replace(
                    '\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n','').replace(
                    '<br/>','').strip()

                if re.search("▶", body):
                    body = body[:re.search("▶", body).start()]

                parsed = (urlparse(newslink)).query.split('&')
                oid = parsed[1]
                aid = parsed[2]

                oidno = re.search(re.compile(r'[^oid=]{3}'), oid).group(0)
                aidno = re.search(re.compile(r'[^aid=]{10}'), aid).group(0)

                sumurl = 'https://tts.news.naver.com/article/'+oidno+'/'+aidno+'/summary'
                sumresp = download(sumurl)
                if sumresp.status_code!=200:
                    continue
                json = sumresp.json()
                #요약
                summary = json['summary'].replace('<br/>','')

                cur.execute('INSERT INTO navernews VALUES(NULL,?,?,?,?,?)',[title,date,body,summary,newslink])
                conn.commit()

20180731 0
20180731 1
20180731 2
20180731 3
20180731 4
20180731 5
20180801 0
20180801 1
20180801 2
20180801 3
20180801 4
20180801 5
20180802 0
20180802 1
20180802 2
20180802 3
20180802 4
20180802 5
20180803 0
20180803 1
20180803 2
20180803 3
20180803 4
20180803 5
20180804 0
20180804 1
20180804 2
20180804 3
20180804 4
20180804 5
20180805 0
20180805 1
20180805 2
20180805 3
20180805 4
20180805 5
20180806 0
20180806 1
20180806 2
20180806 3
20180806 4
20180806 5
20180807 0
20180807 1
20180807 2
20180807 3
20180807 4
20180807 5
20180808 0
20180808 1
20180808 2
20180808 3
20180808 4
20180808 5
20180809 0
20180809 1
20180809 2
20180809 3
20180809 4
20180809 5
20180810 0
20180810 1
20180810 2
20180810 3
20180810 4
20180810 5
20180811 0
20180811 1
20180811 2
20180811 3
20180811 4
20180811 5
20180812 0
20180812 1
20180812 2
20180812 3
20180812 4
20180812 5
20180813 0
20180813 1
20180813 2
20180813 3
20180813 4
20180813 5
20180814 0
20180814 1
20180814 2
20180814 3
20180814 4
20180814 5
20180815 0

KeyboardInterrupt: 

In [2]:
import pandas as pd
pd.to_datetime('20200801', format='%Y%m%d')

Timestamp('2020-08-01 00:00:00')

In [3]:
from datetime import datetime, timedelta 

start_date = datetime.strptime('20190425', '%Y%m%d') 
end_date = datetime.today()
str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

In [4]:
str_date_list[:3]

['20190425', '20190426', '20190427']

In [5]:
import sqlite3

conn = sqlite3.connect('news.db')
cur = conn.cursor()

In [6]:
for j in str_date_list:
    for i in range(6):
        print(j,i)
        url = 'https://news.naver.com/main/ranking/popularDay.nhn'

        params = {
            'rankingType': 'popular_day',
        }

        params['sectionId'] = '10'+str(i)
        params['date']=j

        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }

        resp = download(url, params=params, headers=headers)
        if resp.status_code!=200:
            continue
        dom = BeautifulSoup(resp.content, 'lxml')
        
        
        #뉴스링크 수집
        urlst = []
        for _ in dom.find_all('div',{'class':'ranking_headline'}):
            if _.a['href'] not in urlst:
                urlst.append(_.a['href'])
        
        for _ in urlst:
            newslink = urljoin(url, _)
            resp = download(newslink, headers=headers)
            if resp.status_code!=200:
                continue
            dom = BeautifulSoup(resp.content, 'html.parser')
            if dom.find_all('a',{'class':'media_end_head_autosummary_button _toggle_btn nclicks(sum_summary)'}) != []:
                #제목
                title = dom.find('h3', {'id':'articleTitle'}).text.strip()
                #기사작성일
                date = dom.find_all('span', {'class':'t11'})[0].text.strip()
                #본문
                body = dom.find_all('div',{'id':'articleBodyContents'})[0].text.replace(
                    '\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n','').replace(
                    '<br/>','').strip()

                if re.search("▶", body):
                    body = body[:re.search("▶", body).start()]

                parsed = (urlparse(newslink)).query.split('&')
                oid = parsed[1]
                aid = parsed[2]

                oidno = re.search(re.compile(r'[^oid=]{3}'), oid).group(0)
                aidno = re.search(re.compile(r'[^aid=]{10}'), aid).group(0)

                sumurl = 'https://tts.news.naver.com/article/'+oidno+'/'+aidno+'/summary'
                sumresp = download(sumurl)
                if sumresp.status_code!=200:
                    continue
                json = sumresp.json()
                #요약
                summary = json['summary'].replace('<br/>','')

                cur.execute('INSERT INTO navernews VALUES(NULL,?,?,?,?,?)',[title,date,body,summary,newslink])
                conn.commit()

20190425 0
20190425 1
20190425 2
20190425 3
20190425 4
20190425 5
20190426 0
20190426 1
20190426 2
20190426 3
20190426 4
20190426 5
20190427 0
20190427 1
20190427 2
20190427 3
20190427 4
20190427 5
20190428 0
20190428 1
20190428 2
20190428 3
20190428 4
20190428 5
20190429 0
20190429 1
20190429 2
20190429 3
20190429 4
20190429 5
20190430 0
20190430 1
20190430 2
20190430 3
20190430 4
20190430 5
20190501 0
20190501 1
20190501 2
20190501 3
20190501 4
20190501 5
20190502 0
20190502 1
20190502 2
20190502 3
20190502 4
20190502 5
20190503 0
20190503 1
20190503 2
20190503 3
20190503 4
20190503 5
20190504 0
20190504 1
20190504 2
20190504 3
20190504 4
20190504 5
20190505 0
20190505 1
20190505 2
20190505 3
20190505 4
20190505 5
20190506 0
20190506 1
20190506 2
20190506 3
20190506 4
20190506 5
20190507 0
20190507 1
20190507 2
20190507 3
20190507 4
20190507 5
20190508 0
20190508 1
20190508 2
20190508 3
20190508 4
20190508 5
20190509 0
20190509 1
20190509 2
20190509 3
20190509 4
20190509 5
20190510 0

AttributeError: 'Response' object has no attribute 'requests'

In [6]:
import pandas as pd
pd.to_datetime('20200801', format='%Y%m%d')

from datetime import datetime, timedelta 

start_date = datetime.strptime('20200105', '%Y%m%d') 
end_date = datetime.today()
str_date_list = [] 
while start_date.strftime('%Y%m%d') != end_date.strftime('%Y%m%d'): 
    str_date_list.append(start_date.strftime('%Y%m%d')) 
    start_date += timedelta(days=1)

In [7]:
str_date_list[:3]

['20200105', '20200106', '20200107']

In [8]:
import sqlite3

conn = sqlite3.connect('news.db')
cur = conn.cursor()

In [9]:
for j in str_date_list:
    for i in range(6):
        print(j,i)
        url = 'https://news.naver.com/main/ranking/popularDay.nhn'

        params = {
            'rankingType': 'popular_day',
        }

        params['sectionId'] = '10'+str(i)
        params['date']=j

        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }

        resp = download(url, params=params, headers=headers)
        if resp.status_code!=200:
            continue
        dom = BeautifulSoup(resp.content, 'lxml')
        
        
        #뉴스링크 수집
        urlst = []
        for _ in dom.find_all('div',{'class':'ranking_headline'}):
            if _.a['href'] not in urlst:
                urlst.append(_.a['href'])
        
        for _ in urlst:
            newslink = urljoin(url, _)
            resp = download(newslink, headers=headers)
            if resp.status_code!=200:
                continue
            dom = BeautifulSoup(resp.content, 'html.parser')
            if dom.find_all('a',{'class':'media_end_head_autosummary_button _toggle_btn nclicks(sum_summary)'}) != []:
                #제목
                title = dom.find('h3', {'id':'articleTitle'}).text.strip()
                #기사작성일
                date = dom.find_all('span', {'class':'t11'})[0].text.strip()
                #본문
                body = dom.find_all('div',{'id':'articleBodyContents'})[0].text.replace(
                    '\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n','').replace(
                    '<br/>','').strip()

                if re.search("▶", body):
                    body = body[:re.search("▶", body).start()]

                parsed = (urlparse(newslink)).query.split('&')
                oid = parsed[1]
                aid = parsed[2]

                oidno = re.search(re.compile(r'[^oid=]{3}'), oid).group(0)
                aidno = re.search(re.compile(r'[^aid=]{10}'), aid).group(0)

                sumurl = 'https://tts.news.naver.com/article/'+oidno+'/'+aidno+'/summary'
                sumresp = download(sumurl)
                if sumresp.status_code!=200:
                    continue
                json = sumresp.json()
                #요약
                summary = json['summary'].replace('<br/>','')

                cur.execute('INSERT INTO navernews VALUES(NULL,?,?,?,?,?)',[title,date,body,summary,newslink])
                conn.commit()

20200105 0
20200105 1
20200105 2
20200105 3
20200105 4
20200105 5
20200106 0
20200106 1
20200106 2
20200106 3
20200106 4
20200106 5
20200107 0
20200107 1
20200107 2
20200107 3
20200107 4
20200107 5
20200108 0
20200108 1
20200108 2
20200108 3
20200108 4
20200108 5
20200109 0
20200109 1
20200109 2
20200109 3
20200109 4
20200109 5
20200110 0
20200110 1
20200110 2
20200110 3
20200110 4
20200110 5
20200111 0
20200111 1
20200111 2
20200111 3
20200111 4
20200111 5
20200112 0
20200112 1
20200112 2
20200112 3
20200112 4
20200112 5
20200113 0
20200113 1
20200113 2
20200113 3
20200113 4
20200113 5
20200114 0
20200114 1
20200114 2
20200114 3
20200114 4
20200114 5
20200115 0
20200115 1
20200115 2
20200115 3
20200115 4
20200115 5
20200116 0
20200116 1
20200116 2
20200116 3
20200116 4
20200116 5
20200117 0
20200117 1
20200117 2
20200117 3
20200117 4
20200117 5
20200118 0
20200118 1
20200118 2
20200118 3
20200118 4
20200118 5
20200119 0
20200119 1
20200119 2
20200119 3
20200119 4
20200119 5
20200120 0

In [10]:
cur.close()
conn.close()